# Source

In [1]:
import pandas as pd
import numpy as np
import nltk
#đọc data
df = pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/AML/main/lab-04/spam_detection.csv")
df.head()

,Text,Label,y
0,"Go until jurong point, crazy.. Available only ...",ham,0
1,Ok lar... Joking wif u oni...,ham,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,1
3,U dun say so early hor... U c already then say...,ham,0
4,"Nah I don't think he goes to usf, he lives aro...",ham,0


In [2]:
nltk.download('punkt')
texts = df["Text"].to_list()
texts = [text.lower() for text in texts ]           # chuyển các đoạn text thành chữ thường (word embedding chỉ cho chữ thường)
tokenized_texts = [nltk.tokenize.word_tokenize(text) for text in texts]    # tách câu thành một list các từ

print(tokenized_texts[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['go', 'until', 'jurong', 'point', ',', 'crazy..', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'cine', 'there', 'got', 'amore', 'wat', '...']


In [3]:
#Tải bộ weights Glove
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-11-20 13:24:06--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-11-20 13:24:06--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-11-20 13:24:07--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [4]:
#Giải Nén
! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [5]:
## không cần hiểu đống này lắm đâu
import io
import numpy as np
def load_word_embeddings(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    vocab, matrix = [], []
    i=0
    for line in fin:
        tokens = line.rstrip().split(' ')
        vocab.append(tokens[0])
        matrix.append(list(map(float, tokens[1:])))
    return vocab, np.asarray(matrix)

In [6]:
#vocab là bộ từ điển các từ có trong Glove
#matrix chứa các vector biểu diễn các từ 
##Ta chọn bộ weighs có số chiều là 100
vocab, matrix = load_word_embeddings("glove.6B.300d.txt")
matrix.shape

(400000, 300)

In [7]:
## Gán các mã
__PADDED_INDEX__ = 0    # mã dùng cho các vị trí chỉ có tính nối dài cho cùng kích thước
__UNKNOWN_WORD__ = 1    # mã cho những từ không có trong embedding

In [8]:
# Tạo một dictionary, có nhiệm vụ là một ánh xạ từ ảnh sang mã số, mã số được bắt đầu từ 2 vì số 0 và 1 được dành cho trường hợp đặc biệt
word_to_index = {word: index+2 for index, word in enumerate(vocab)}

In [9]:
# Do do mã số được bắt đầu từ 2, nên cần thêm 2 vector vào đàu ma trận
embedding_matrix = np.pad(matrix, ((2,0),(0,0)), mode='constant', constant_values =0.0)
print(embedding_matrix)

[[ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 0.04656    0.21318   -0.0074364 ...  0.0090611 -0.20989    0.053913 ]
 ...
 [ 0.075713  -0.040502   0.18345   ...  0.21838    0.30967    0.43761  ]
 [ 0.81451   -0.36221    0.31186   ...  0.075486   0.28408   -0.17559  ]
 [ 0.429191  -0.296897   0.15011   ...  0.28975    0.32618   -0.0590532]]


In [10]:
## Bây giờ ta sẽ chuyển data spam dection thành các mã số
import tensorflow as tf

X = []
for text in tokenized_texts:
    cur_text_indices = []
    for word in text:
        if word in word_to_index:
            cur_text_indices.append(word_to_index[word])    ## map từ word sang index
        else:
            cur_text_indices.append(__UNKNOWN_WORD__)       ## gán unknown cho từ không có trong bộ glove
    X.append(cur_text_indices)

## pad data cho có cùng chiều dài
X = tf.keras.preprocessing.sequence.pad_sequences(sequences=X,       # sequences: list các câu có độ dài không bằng nhau
                                                  padding='post')    # vị trí pad là 'pre' (trước) hoặc 'post' (sau)

y = df['y'].values   ## Label của bài toán, 0 là không phải spam, 1 là spam

In [11]:
## Chia data
from sklearn.model_selection import train_test_split
X_1, X_test, y_1, y_test = train_test_split(X, y, test_size= 0.2, random_state =0)
X_train, X_valid, y_train, y_valid = train_test_split(X_1, y_1, test_size= 0.1, random_state =0)

1. Hãy dùng bộ FastText 300d để làm embedding cho model
2. Hãy tìm hiểu cách dùng của convolution 1D trong keras để train mô hình dưới đây

In [12]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, Conv1D, GlobalAveragePooling1D
from tensorflow.keras.models import Model

inputs = Input(shape=(None,))                   ## None biểu thị kích thước không xác định của câu

embed = Embedding(input_dim=embedding_matrix.shape[0],   ## Khai báo kích thước của vocab
                 output_dim=embedding_matrix.shape[1],   ## Khai báo kích thước của embedding
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  ## Khởi tạo cho embedding bằng ma trận có sẵn
                  trainable=False,                       ## Không cần thiết train embedding
                 mask_zero=True)(inputs)                 ## zero_mask: những vị trí có giá trị 0 không được tính toán, vì đó là giá trị thêm vào cho đủ độ dài mà thôi
                                                         ##  (__PADDED_INDEX__ gán bằng 0)

#conv1D = Conv1D(filters=256, kernel_size=(1, 1), strides=(1,1), padding='same')(embed)      ## return_sequences: LSTM trả về toàn bộ  hay là trả về hidden_state cuối cùng
conv1D = Conv1D(filters=100, kernel_size=2, strides=2, padding='same')(embed) 

global_average_pooling1d = GlobalAveragePooling1D()(conv1D)
dense = Dense(units=2, activation='softmax')(global_average_pooling1d)
model = Model(inputs=inputs,
              outputs=dense)

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 300)         120000600 
                                                                 
 conv1d (Conv1D)             (None, None, 100)         60100     
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 120,060,902
Trainable params: 60,302
Non-trainable params: 120,000,600
__________________________________________

In [13]:
#compile model
#model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=5e-5), loss=tf.keras.losses.sparse_categorical_crossentropy ,metrics=["accuracy"])

history=model.fit(x=X_train, y=y_train,
                  epochs=20,
                  verbose=1,
                  validation_data=(X_valid,y_valid))

#Xem độ chính xác trên tập test
evaluation=model.evaluate(X_test, y_test)
print("Model có độ chính xác là {} và loss là {}".format(evaluation[1], evaluation[0]))

Epoch 1/20
126/126 [==============================] - 31s 9ms/step - loss: 0.3384 - accuracy: 0.8881 - val_loss: 0.1842 - val_accuracy: 0.9507
Epoch 2/20
126/126 [==============================] - 1s 7ms/step - loss: 0.1343 - accuracy: 0.9574 - val_loss: 0.1182 - val_accuracy: 0.9552
Epoch 3/20
126/126 [==============================] - 1s 7ms/step - loss: 0.0955 - accuracy: 0.9669 - val_loss: 0.1056 - val_accuracy: 0.9619
Epoch 4/20
126/126 [==============================] - 1s 7ms/step - loss: 0.0824 - accuracy: 0.9733 - val_loss: 0.1011 - val_accuracy: 0.9596
Epoch 5/20
126/126 [==============================] - 1s 7ms/step - loss: 0.0745 - accuracy: 0.9756 - val_loss: 0.0996 - val_accuracy: 0.9686
Epoch 6/20
126/126 [==============================] - 1s 7ms/step - loss: 0.0650 - accuracy: 0.9811 - val_loss: 0.0941 - val_accuracy: 0.9664
Epoch 7/20
126/126 [==============================] - 1s 7ms/step - loss: 0.0603 - accuracy: 0.9818 - val_loss: 0.0989 - val_accuracy: 0.9709
Epoch

In [14]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, Conv1D, GlobalAveragePooling1D
from tensorflow.keras.models import Model

inputs = Input(shape=(None,))                   ## None biểu thị kích thước không xác định của câu

embed = Embedding(input_dim=embedding_matrix.shape[0],   ## Khai báo kích thước của vocab
                 output_dim=embedding_matrix.shape[1],   ## Khai báo kích thước của embedding
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  ## Khởi tạo cho embedding bằng ma trận có sẵn
                  trainable=True,                       ## Không cần thiết train embedding
                 mask_zero=True)(inputs)                 ## zero_mask: những vị trí có giá trị 0 không được tính toán, vì đó là giá trị thêm vào cho đủ độ dài mà thôi
                                                         ##  (__PADDED_INDEX__ gán bằng 0)

#conv1D = Conv1D(filters=256, kernel_size=(1, 1), strides=(1,1), padding='same')(embed)      ## return_sequences: LSTM trả về toàn bộ  hay là trả về hidden_state cuối cùng
#conv1D = Conv1D(filters=100, kernel_size=2, padding='same')(embed) 
conv1D = Conv1D(filters=100, kernel_size=2, strides=2, padding='same')(embed) 

global_average_pooling1d = GlobalAveragePooling1D()(conv1D)
dense = Dense(units=2, activation='softmax')(global_average_pooling1d)
model_1 = Model(inputs=inputs,
              outputs=dense)

model_1.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 300)         120000600 
                                                                 
 conv1d_1 (Conv1D)           (None, None, 100)         60100     
                                                                 
 global_average_pooling1d_1   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 2)                 202       
                                                                 
Total params: 120,060,902
Trainable params: 120,060,902
Non-trainable params: 0
_____________________________________________

In [16]:
#compile model
#model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=5e-5), loss=tf.keras.losses.sparse_categorical_crossentropy ,metrics=["accuracy"])

history=model_1.fit(x=X_train, y=y_train,
                  epochs=20,
                  verbose=1,
                  validation_data=(X_valid,y_valid))

#Xem độ chính xác trên tập test
evaluation=model_1.evaluate(X_test, y_test)
print("Model có độ chính xác là {} và loss là {}".format(evaluation[1], evaluation[0]))

Epoch 1/20
126/126 [==============================] - 13s 105ms/step - loss: 0.0022 - accuracy: 0.9995 - val_loss: 0.1078 - val_accuracy: 0.9843
Epoch 2/20
126/126 [==============================] - 13s 104ms/step - loss: 0.0016 - accuracy: 0.9998 - val_loss: 0.1177 - val_accuracy: 0.9865
Epoch 3/20
126/126 [==============================] - 13s 104ms/step - loss: 0.0024 - accuracy: 0.9995 - val_loss: 0.1105 - val_accuracy: 0.9843
Epoch 4/20
126/126 [==============================] - 13s 105ms/step - loss: 0.0025 - accuracy: 0.9993 - val_loss: 0.1135 - val_accuracy: 0.9753
Epoch 5/20
126/126 [==============================] - 13s 105ms/step - loss: 0.0022 - accuracy: 0.9998 - val_loss: 0.1072 - val_accuracy: 0.9843
Epoch 6/20
126/126 [==============================] - 13s 104ms/step - loss: 0.0037 - accuracy: 0.9993 - val_loss: 0.0950 - val_accuracy: 0.9821
Epoch 7/20
126/126 [==============================] - 13s 104ms/step - loss: 0.0017 - accuracy: 0.9998 - val_loss: 0.1051 - val_ac

Xét 2 trường hợp trainable là True và False trong lớp embedding và đưa ra so sánh

Sau khi chạy model với 2 trường hợp trainable là True và False thì ta thấy khi xét True thì thời gian chạy của model lâu hơn khi ta để là False vì khi ta đặt là True 
model sẽ train bộ FastText ban đầu.
Còn về accuracy của 2 cách chạy cho ra tương đối xấp xỉ nhau còn loss thì cao hơn khi ta để True